In [166]:
import pandas as pd
import numpy as np
import string
import re
from IPython.display import clear_output
import requests
from nltk.corpus import stopwords
#nltk.download("stopwords")

import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'nltk'

In [2]:
def update_progress(progress):
    bar_length = 50
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

---

In [3]:
URL = "https://de.wikipedia.org/w/api.php"

In [5]:
# request excellent arictles from german wikipedia via wiki api (10 at a time)
S = requests.Session()

params = {
    "action": "query",
    "prop": "revisions",
    "rvprop": "content",
    "rvslots": "*",
    "format": "json",
    "formatversion": 2,
    "srsearch": "incategory:Wikipedia:Exzellent",
    "list": "search",
    "sroffset": 0
}

response = S.get(url = URL, params = params)
data = response.json()

# get ids from excellent articles
ids = []

for entry in data["query"]["search"]:
    ids.append(entry["pageid"])

"""while data.get("continue"):
    params.update({"sroffset": data["continue"]["sroffset"]})
    
    #print("\n%s" % (PARAMS))
    response = S.get(url = URL, params = params)
    data = response.json()
    
    for entry in data["query"]["search"]:
        ids.append(entry["pageid"])"""

print("Anzahl gesammelter Exzellenter Artikel: %s" %(len(ids)))

#if DATA['query']['search'][0]['title'] == SEARCHPAGE:
#    print("Your search page '" + SEARCHPAGE + "' exists on English Wikipedia")

Anzahl gesammelter Exzellenter Artikel: 10


In [6]:
# request data to every excellent article in german wikipedia via wikipedia api using pageid
params = {
    "action": "query",
    "prop": "revisions",
    "rvprop": "content",
    "rvslots": "*",
    "format": "json",
    "formatversion": 2,
    "pageids": 0
}

data = pd.DataFrame()
content = {}

for ix, id in enumerate(ids):
    update_progress(ix / len(ids))
    
    params.update({"pageids": id})
    response = S.get(url = URL, params = params)
    page = response.json()
    
    content.update({id: page["query"]["pages"][0]["revisions"][0]["slots"]["main"]["content"]})

Progress: [#############################################-----] 90.0%


In [7]:
# save requested data in df
df = pd.DataFrame(content.items())
df = df.rename({0: "pageid", 1:"content"}, axis = 1)

In [8]:
df

,pageid,content
0,1428,"[[Datei:Danimarca XIII secolo, plinio historia..."
1,2677,{{Begriffsklärungshinweis}}\n{{Infobox Staat\n...
2,1200964,{{Dieser Artikel|behandelt den Staat. Zu weite...
3,880316,{{Begriffsklärungshinweis}}\n{{Infobox Staat\n...
4,2391,{{Dieser Artikel|behandelt das Land – zu ander...
5,16565,{{Dieser Artikel|behandelt die Stadt Frankfurt...
6,3221050,{{Begriffsklärungshinweis}}\n{{Infobox Ort in ...
7,2996,{{Dieser Artikel|behandelt die Großstadt in Sa...
8,290,{{Infobox Staat\n|NAME-AMTSSPRACHE =...
9,1084,{{Begriffsklärungshinweis}}\n{{Infobox Gemeind...


# save data to csv for faster loading
df.to_csv("./2783_excellent_article.csv")

In [12]:
contents = df["content"]

In [17]:
first_page = contents[0]

In [19]:
first_page = first_page.lower()

In [20]:
first_page

'[[datei:danimarca xiii secolo, plinio historia naturalis.jpg|mini|die \'\'[[naturalis historia]]\'\' des [[plinius der ältere|älteren plinius]] in einer reich illustrierten ausgabe des 13.&nbsp;jahrhunderts]]\n[[datei:larousse secxix.jpg|mini|\'\'nouveau [[éditions larousse|larousse]] illustré\'\', 1897–1904]]\n[[datei:2020 11 04 004 meyers konversations lexikon.jpg|mini|[[meyers konversations-lexikon]], 4. auflage, bände 1–16 (1885–1890)]]\n[[datei:bertelsmann lexikothek.jpg|mini| \'\'[[bertelsmann lexikothek]]\'\' in 26 bänden, in der auflage von 1983]]\neine \'\'\'enzyklopädie\'\'\' ({{audio|de-enzyklopädie.ogg}}), früher auch aus dem [[französische sprache|französischen]]: \'\'\'encyclopédie\'\'\' (von {{grcs|ἐγκύκλιος παιδεία|enkýklios paideía|de=kreis von wissenschaften und künsten, welche jeder freie grieche in der jugend treiben musste, bevor er ins bürgerliche leben eintrat oder sich einem besondern studium widmete}}, d.&nbsp;h. was wir heute „grundausbildung, allgemeine erzi

In [164]:
# remove "&nbsp"
test_page = re.sub(r"\&nbsp", "", first_page)
# remove urls
# source: url_extract_pattern from https://uibakery.io/regex-library/url-regex-python
url_extract_pattern = "https?:\\/\\/(?:www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)"
test_page = re.sub(url_extract_pattern, '', test_page)
# remvoe "\n"
test_page = re.sub(r"[^ ]*\n", "", test_page)
# remove file names with commom endings with 4 or 3 digits
test_page = re.sub(r"[^ ]*\..{4}|[^ ]*\..{3}", "", test_page)
# remove any refs
test_page = re.sub(r"[^ ]*ref", "", test_page)
# remove -
test_page = re.sub(r"-", "", test_page)

test_page = test_page.translate(str.maketrans('', '', string.punctuation))

In [165]:
test_page

'dateidanimarca xiii secolo plinio historia minidie naturalis historia des plinius der ältereälteren plinius in einer reich illustrierten ausgabe des dateilarousse mininouveau éditions larousselarousse illustré datei2020 11 04 004 meyers konversations minimeyers konversationslexikon lage bände 1–16 dateibertelsmann mini bertelsmann lexikothek in 26 bänden in der auflage von eine enzyklopädie  früher auch aus dem französische sprachefranzösischen encyclopédie von grcsἐγκύκλιος παιδείαenkýklios paideíadekreis von wissenschaften und künsten welche jeder freie grieche in der jugend treiben musste bevor er ins bürgerliche leben eintrat oder sich einem besondern studium widmete  wir heute „grundausbildung allgemeine erziehung allgemeinbildung“ literatur autorwilhelm pape max sengebusch itelhandwörterbuch der griechischen sprache lage ruck verlagvieweg  sohn ortbraunschweig datum1914 online abruf20210118 kommentarzur bei pape nur abgekürzten maskulinen form langgrcἐγκύκλιοςenkýkliosdekreisför

In [ ]:
test_page